<span style="color: #0000ff;">Database</span> Transformation (OLTP <span style="color: #0000ff;">to</span> OLAP)

In [2]:
SELECT * 
INTO DimCategories
FROM ecommerce.dbo.categories;

: Msg 40515, Level 15, State 1, Line 16
Reference to database and/or server name in 'ecommerce.dbo.categories' is not supported in this version of SQL Server.

Total execution time: 00:00:00.007

In [4]:
ALTER TABLE DimCategories 
ADD primary key (categoryID);

Commands completed successfully.

Total execution time: 00:00:00.021

In [5]:
SELECT * 
INTO DimPaymentMethods
FROM ecommerce.dbo.payment_methods;

(5 rows affected)

Total execution time: 00:00:00.004

In [7]:
ALTER TABLE DimPaymentMethods 
ADD primary key (PaymentID);

Commands completed successfully.

Total execution time: 00:00:00.006

In [8]:
SELECT *
INTO DimProducts
FROM ecommerce.dbo.products;

(120 rows affected)

Total execution time: 00:00:00.005

In [10]:
ALTER TABLE DimProducts
ADD primary key (product_id);

Commands completed successfully.

Total execution time: 00:00:00.009

In [12]:
ALTER TABLE DimProducts 
ADD foreign key (category_id) references DimCategories(categoryID);

Commands completed successfully.

Total execution time: 00:00:00.011

In [13]:
SELECT * 
INTO DimCustomers
FROM ecommerce.dbo.customers;

(50 rows affected)

Total execution time: 00:00:00.004

In [15]:
ALTER TABLE DimCustomers 
ADD primary key (customer_id);

Commands completed successfully.

Total execution time: 00:00:00.011

In [3]:
SELECT * 
INTO DimLocations
FROM ecommerce.dbo.locations;

(50 rows affected)

Total execution time: 00:00:00.007

In [4]:
ALTER TABLE DimLocations 
ADD primary key (location_id);

Commands completed successfully.

Total execution time: 00:00:00.015

In [16]:
WITH TotalSales AS (
    SELECT od.order_id, SUM(p.price * od.quantity) AS TotalOrderSales
    FROM ecommerce.dbo.orderDetails AS od
    JOIN ecommerce.dbo.products AS p ON od.product_id = p.product_id
    GROUP BY od.order_id
)
SELECT 
    o.orders_id, 
    o.date, 
    o.customer_id,
    o.payment_method_id, 
    od.product_id, 
    od.quantity,
    (p.price * od.quantity) AS SalesAmount,
    ts.TotalOrderSales
INTO FactTable
FROM ecommerce.dbo.orders o 
JOIN ecommerce.dbo.orderDetails od 
ON o.orders_id = od.order_id
JOIN ecommerce.dbo.products AS p ON od.product_id = p.product_id
JOIN TotalSales ts ON ts.order_id = o.orders_id;


(443 rows affected)

Total execution time: 00:00:00.022

In [18]:
ALTER TABLE FactTable 
ADD factTableID SMALLINT IDENTITY(1, 1) PRIMARY KEY;

ALTER TABLE FactTable ADD 
foreign key (customer_id) references DimCustomers(customer_id);


ALTER TABLE FactTable 
ADD foreign key (product_id) references DimProducts(product_id);

: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'factTableID' in table 'FactTable' is specified more than once.

Total execution time: 00:00:00.001

In [19]:
ALTER TABLE FactTable 
ADD foreign key (payment_method_id) references DimPaymentMethods(PaymentID);

Commands completed successfully.

Total execution time: 00:00:00.006

In [ ]:
-- Indexing --
CREATE INDEX idx_fact_customerid ON FactTable(customer_id);
CREATE INDEX idx_fact_paymentmethodid ON FactTable(payment_method_id);
CREATE INDEX idx_fact_productid ON FactTable(product_id);

In [21]:
-- Data Quality Scripts --
SELECT * 
FROM FactTable
WHERE SalesAmount < 0 OR TotalOrderSales < 0;

(0 rows affected)

Total execution time: 00:00:00.004

orders_id,date,customer_id,payment_method_id,product_id,quantity,SalesAmount,TotalOrderSales,factTableID
